In [1]:
import field

mask_field = field.Place_1
draw_field = field.Place_1

START_POS = (1, 1)
for i in range(len(mask_field)):
    for j in range(len(mask_field[0])):
        if mask_field[i][j] == 'a':
            START_POS = (i, j)
            break
        
END_POS = (1, 1)
for i in range(len(mask_field)):
    for j in range(len(mask_field[0])):
        if mask_field[i][j] == 'b':
            END_POS = (i, j)
            break
        
SIDE_MOVE: int = 10
DIAG_MOVE: int = 14
MAX_X: int = len(mask_field) - 1
MAX_Y: int = len(mask_field[0]) - 1

In [2]:

class Cell:
    def __init__(self, parent: tuple[int], pos: tuple[int], g: int):
        self.parent: tuple[int] = parent
        self.pos = pos
        self.g = g
        self.h = self.count_h()
        self.cost = 0
        self.count_cost()
        
    def count_h(self) -> int:
        return abs(self.pos[0] - END_POS[0]) + abs(self.pos[1] - END_POS[1])
    
    def count_cost(self):
        self.cost = self.g + self.h
        
        
def draw():
    for cell in closed_cells:
        pos = cell.pos
        draw_field[pos[0]][pos[1]] = cell.cost

open_cells: list[Cell] = list()
open_pos: list[tuple[int]] = list()

open_cells.append(Cell(START_POS, START_POS, 0))

closed_cells: list[Cell] = list()
closed_pos: list[tuple[int]] = list()
blocks: list[tuple[int]] = list()
for x in range(len(mask_field)):
    for y in range(len(mask_field[0])):
        if mask_field[x][y] == 'x':
            closed_pos += [(x, y)]
            blocks += [(x, y)]
        

In [3]:
from PIL import Image, ImageDraw, ImageFont

def ShowPlace(Place):
    n = 0
    for i in Place:
        n += 1
        m = 0
        for j in i:
            m += 1
            if j == 'x':
                re_PL.rectangle((m*100-100, n*100-100, m*100, n*100),
                                fill='black', outline=(255, 255, 255))
            elif j == 'a' or j == 'b':
                re_PL.rectangle((m*100-100, n*100-100, m*100, n*100),
                                fill='yellow', outline=(0, 0, 0))
                re_PL.text((m*100-80, n*100-80), j, fill='black', font=front)
            else:
                re_PL.rectangle((m*100-100, n*100-100, m*100, n*100),
                                fill='white', outline=(0, 0, 0))
                re_PL.text((m*100-90, n*100-90), str(j),
                           fill='black', font=front)
                
PL = Image.new('RGB', (1600, 1600), (255, 255, 255))
re_PL = ImageDraw.Draw(PL)
front = ImageFont.truetype("arial.ttf", 50)

In [4]:
while True:
    open_cells.sort(key=lambda x: x.cost, reverse=True)
    open_pos = list(map(lambda x: x.pos, open_cells))
    
    cell = open_cells[-1]
    cell_pos = cell.pos
    
    if cell_pos == END_POS:
        closed_cells.append(open_cells.pop())
        break
    
    for i in range(-1, 2):
        for j in range(-1, 2):
            new_pos = (cell_pos[0] + i, cell_pos[1] + j)
            
            if (0 <= new_pos[0] <= MAX_X) and (0 <= new_pos[1] <= MAX_Y) and (new_pos != cell_pos) and (new_pos not in closed_pos):
                
                if (i != 0 and j == 0) or (i == 0 and j != 0): move = SIDE_MOVE
                else: move = DIAG_MOVE
                
                # Если запрещены срезы (если не запрещены -- все плохо, но никому об этом не говорить)
                if move == DIAG_MOVE:
                    if (cell_pos[0]+i, cell_pos[1]) in blocks or (cell_pos[0], cell_pos[1]+j) in blocks: continue

                if new_pos not in open_pos:
                    open_cells.append(Cell(cell_pos, new_pos, cell.g + move))
                elif new_pos in open_pos:
                    opened_cell_idx = open_pos.index(new_pos)
                    opened_cell = open_cells[opened_cell_idx]
                    
                    new_g = cell.g + move
                    
                    if new_g < opened_cell.g:
                        open_cells[opened_cell_idx] = Cell(cell_pos, new_pos, new_g)
    
    closed_pos.append(cell_pos)
    open_cells.sort(key=lambda x: x.cost, reverse=True)
    closed_cells.append(open_cells.pop())
    
    # print(len(open_cells))
    # draw()
    # break
    

In [5]:
# open_cells.append(Cell((1, 1), (1, 1), 0))
# open_cells.sort(key=lambda x: x.cost, reverse=True)
# open_cells = list(map(lambda x: x.cost, open_cells))
# open_cells

In [6]:
draw()
ShowPlace(draw_field)
# PL.show()

In [7]:
def draw_path(cells: list[Cell]):
    now_cell = cells[-1]
    cells_pos = list(map(lambda x: x.pos, cells))
    
    font = ImageFont.truetype("arial.ttf", 50)
    x, y = now_cell.pos[0], now_cell.pos[1]
    re_PL.rectangle((y*100, x*100, y*100+100, x*100+100),
                            fill='yellow', outline=(0, 0, 0))
    re_PL.text((y*100+20, x*100+20), 'b', fill='black', font=front)
    while True:
        prev_coord = now_cell.parent
        now_cell = cells[cells_pos.index(prev_coord)]
        
        # print(now_cell.pos)
        
        x, y = now_cell.pos[0], now_cell.pos[1]
        
        re_PL.rectangle((y*100, x*100, y*100+100, x*100+100),
                                fill='blue', outline=(0, 0, 0))
        re_PL.text((y*100+10, x*100+10), str(draw_field[x][y]), fill='black', font=font)
        
        if now_cell.pos == START_POS:
            re_PL.rectangle((y*100, x*100, y*100+100, x*100+100),
                            fill='yellow', outline=(0, 0, 0))
            re_PL.text((y*100+20, x*100+20), 'a', fill='black', font=front)
            break
        
        
        
        
        

In [8]:
draw_path(closed_cells)

In [9]:
PL.show()